# Livrable final du projet - Projet Green Graph
*Equipe CesiCDP - Chef de projet : Leila | Opérateurs : Tom, Edwin*</br>
## Sommaire :
    1. Introduction

    PARTIE 1 : Modélisation
    2. Rappel de la modélisation formelle
        2.1. Résumé des hypothèses et représentations
        2.2. Justification du maintien de la modélisation
    3. Méthodes de résolution
        3.1. Choix des algorithmes
        3.2. Description des algorithmes 
        3.3. Complexité des algorithmes 

    PARTIE 2 : Implémentation et exploitation
    4. Implémentation
        4.1. Détails de l’implémentation des algorithmes
        4.2. Cas de test représentatifs
        4.3. Résultats observés sur les tests
    5. Étude expérimentale
        5.1. Méthodologie du plan d’expérience
        5.2. Analyse des performances 
        5.3. Limites observées
        5.4. Perspectives d’amélioration

    6. Plan de Travail et Organisation du Projet
        6.1. Étapes prévues 

    7. Conclusion

    8. Annexes
        8.1. Glossaire
        8.2. Références bibliographiques
        8.3. Annexes techniques 

### 1. Introduction
Dans la continuité de notre premier livrable, ce document final a pour objectif de présenter l'ensemble de la démarche menée autour du projet Green Graph, de la modélisation initiale à l'évaluation expérimentale des solutions implémentées.

Pour rappel, on se questionnait sur comment réduire l’impact environnemental des tournées de livraison tout en optimisant les coûts opérationnels.<br>
En modélisant ce problème à partir du **Problème du Voyageur de Commerce (PVC)** complété avec des contraintes réalistes :
* **Coût ou restriction de passage sur certaines arêtes :** Certaines routes peuvent être plus coûteuses ou interdites (par exemple, travaux ou routes bloquées),
* **Dépendances entre visites :** Une ville ne peut être visitée qu'après en avoir visité une autre (par exemple, une livraison doit précéder une collecte),

nous avons pu définir une approche formelle qui répond aux besoins de planification d'un trajet routier réel.




## Partie 1
### 2. Rappel de la modélisation formelle
##### 2.1. Résumé des hypothèses et représentations
##### 2.2. Justification du maintien de la modélisation

### 3. Méthodes de résolution
##### 3.1. Choix des algorithmes
##### 3.2. Description des algorithmes 
##### 3.3. Complexité des algorithmes 



### 4. Implémentation
##### 4.1. Détails de l’implémentation des algorithmes
##### 4.2. Cas de test représentatifs
##### 4.3. Résultats observés sur les tests


### 5. Étude expérimentale
##### 5.1. Méthodologie du plan d’expérience
##### 5.2. Analyse des performances 
##### 5.3. Limites observées
##### 5.4. Perspectives d’amélioration



### 6. Plan de Travail et Organisation du Projet
##### 6.1. Étapes prévues 


### 7. Conclusion


### 8. Annexes
##### 8.1. Glossaire
##### 8.2. Références bibliographiques
##### 8.3. Annexes techniques 